In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/silver/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-629a8b54-b5da-4bc0-b83a-4e728b6ac2e1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 1551ms :: artifacts dl 26ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-ru

In [2]:
# apagar tudo do hdfs

#to_path = "/projeto/fisico/bronze/stats/"
#to_path_aux = "/projeto/fisico/bronze/players_stats_by_season_full_details/"
#to_path_aux_teams = "/projeto/fisico/bronze/team_abbreviations/"
#to_path_aux_age = "/projeto/fisico/bronze/age/"
#to_path_aux_hist = "/projeto/fisico/bronze/historical/"

#to_path_silver = "/projeto/fisico/silver/stats/"


#client.delete(to_path,True)
#client.delete(to_path_aux,True)
#client.delete(to_path_aux_teams,True)
#client.delete(to_path_aux_age,True)
#client.delete(to_path_aux_hist,True)

#client.delete(to_path_silver,True)

In [2]:
# declaracao de todos os caminhos

from_path = "../projeto/bronze/stats/"
from_path_aux = "../projeto/bronze/players_stats_by_season_full_details/"
from_path_aux_teams = "../projeto/bronze/team_abbreviations/"
from_path_aux_age = "../projeto/bronze/age/"
from_path_aux_hist = "../projeto/bronze/historical/"

hdfs_path = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/stats"
hdfs_path_aux = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/players_stats_by_season_full_details"
hdfs_path_aux_teams = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/team_abbreviations"
hdfs_path_aux_age1718 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1718.csv"
hdfs_path_aux_age1819 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1819.csv"
hdfs_path_aux_age1920 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/age/age_1920.csv"
hdfs_path_aux_hist = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/historical"

In [4]:
# upload de bronze

#client.upload(to_path, from_path)
#client.upload(to_path_aux, from_path_aux)
#client.upload(to_path_aux_teams, from_path_aux_teams)
#client.upload(to_path_aux_age, from_path_aux_age)
client.upload(to_path_aux_hist, from_path_aux_hist)

'/projeto/fisico/bronze/historical'

In [3]:
# criacao dos dataframes

dataFrame = spark.read.csv(hdfs_path, header= True)
dataFrame_aux = spark.read.csv(hdfs_path_aux, header= True)
dataFrame_aux_teams = spark.read.csv(hdfs_path_aux_teams, header= True)
dataFrame_aux_age1718 = spark.read.csv(hdfs_path_aux_age1718, header= True)
dataFrame_aux_age1819 = spark.read.csv(hdfs_path_aux_age1819, header= True)
dataFrame_aux_age1920 = spark.read.csv(hdfs_path_aux_age1920, header= True)
dataFrame_aux_hist = spark.read.csv(hdfs_path_aux_hist, header= True)

In [6]:
# visualizacao dos dataframes - alterar confome necessário

dataFrame.printSchema()
dataFrame_aux.printSchema()
dataFrame_aux_teams.printSchema()
#dataFrame.show(truncate=False)
#dataFrame.toPandas()

root
 |-- #: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Tm: string (nullable = true)
 |-- G: string (nullable = true)
 |-- GS: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- PER: string (nullable = true)
 |-- TS%: string (nullable = true)
 |-- 3PAr: string (nullable = true)
 |-- FTr: string (nullable = true)
 |-- ORB%: string (nullable = true)
 |-- DRB%: string (nullable = true)
 |-- TRB%: string (nullable = true)
 |-- AST%: string (nullable = true)
 |-- STL%: string (nullable = true)
 |-- BLK%: string (nullable = true)
 |-- TOV%: string (nullable = true)
 |-- USG%: string (nullable = true)
 |-- blanl: string (nullable = true)
 |-- OWS: string (nullable = true)
 |-- DWS: string (nullable = true)
 |-- WS: string (nullable = true)
 |-- WS/48: string (nullable = true)
 |-- blank2: string (nullable = true)
 |-- OBPM: string (nullable = true)
 |

In [7]:
# visualizacao estatistica dos dataframes - alterar conforme necessário

dataFrame_status = dataFrame.summary()
#dataFrame_aux_status = dataFrame_aux.summary()
#dataFrame_aux_teams_status = dataFrame_aux_teams.summary()

dataFrame_status.toPandas()
#dataFrame_aux_status.toPandas()
#dataFrame_aux_teams_status.toPandas()

22/01/13 19:27:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,#,Year,Player,Pos,Age,Tm,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,count,24691,24624,24624,24624,24616,24624,24624,18233,24138,...,23766,20797,20797,24312,24624,20797,20797,19645,24624,24624
1,mean,12345.0,1992.5949886289798,None,None,26.66440526486838,None,50.83711013645224,23.593374650359237,1209.7203165133815,...,0.7192786333417495,62.18920998220897,147.1994037601577,224.63738071734124,114.85262345679013,39.89705245948935,24.470260133673126,73.93983201832528,116.33922189733593,510.1163499025341
2,stddev,7127.822084386413,17.429594300692777,None,None,3.841891735879667,None,26.49616061972552,28.632386992475652,941.1465751248744,...,0.1418240282611371,67.32488075091283,145.92191156074958,228.19020337713175,135.8639131278369,38.71305272768127,36.93508446335137,67.71380269648333,84.79187270766582,492.92298138266773
3,min,0,1950,A.C. Green,C,18,AND,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,25%,6170.0,1981.0,None,None,24.0,None,27.0,0.0,340.0,...,0.657,12.0,33.0,51.0,19.0,9.0,3.0,18.0,39.0,106.0
5,50%,12344.0,1996.0,None,None,26.0,None,58.0,8.0,1053.0,...,0.743,38.0,106.0,159.0,68.0,29.0,11.0,55.0,109.0,364.0
6,75%,18516.0,2007.0,None,None,29.0,None,75.0,45.0,1971.0,...,0.808,91.0,212.0,322.0,160.0,60.0,29.0,112.0,182.0,778.0
7,max,9999,2017,Zydrunas Ilgauskas,SG-SF,44,WSC,9,9,999,...,1,99,993,997,991,99,99,99,99,999


In [5]:
# edição do dataframe das idades

In [4]:
df1718 = dataFrame_aux_age1718.withColumn(
    "Year",
    lit("2018")
    )
df1819 = dataFrame_aux_age1819.withColumn(
    "Year",
    lit("2019")
    )
df1920 = dataFrame_aux_age1920.withColumn(
    "Year",
    lit("2020")
    )

In [5]:
# junção dos 3

df_aux_age = df1718.union(df1819.union(df1920))
df_aux_age = df_aux_age.drop("Team")
df_aux_age = df_aux_age.withColumnRenamed("Player","Player1")
df_aux_age = df_aux_age.withColumnRenamed("Year","Year1")

In [6]:
# join do df das idades ao aux

In [7]:
# inicio da edição dos dataframes

In [8]:
# adicionar nomes das equipas por extenso

dataFrame = dataFrame.join(dataFrame_aux_teams, dataFrame.Tm == dataFrame_aux_teams.Abbreviation, 'leftouter')

dataFrame_aux = dataFrame_aux.join(dataFrame_aux_teams, dataFrame_aux.Team == dataFrame_aux_teams.Abbreviation, 'leftouter')

In [9]:
# adaptacao do df aux ao df principal

In [10]:
dataFrame_aux = dataFrame_aux.withColumn(    
    "Year",
    dataFrame_aux.Season.substr(8, 4)
)

In [11]:
cond = [dataFrame_aux.Year == df_aux_age.Year1, dataFrame_aux.Player == df_aux_age.Player1]

dataFrame_aux = dataFrame_aux.join(df_aux_age, cond, 'leftouter')

In [12]:
dataFrame_aux.createOrReplaceTempView("dataFrame_aux")

# preparar aux para a juncao
dataFrame_aux = spark.sql(
    """
    SELECT Year,Player,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS,Franchise
    FROM dataFrame_aux
    WHERE (Year == "2018" OR Year == "2019" OR Year == "2020")
    AND League == "NBA"
    AND Stage == "Regular_Season"
    """
)

dataFrame_aux.show()

22/01/13 22:40:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+--------------------+----+----+---+------+---+----+---+---+---+---+---+---+---+---+----+---+---+---+----+--------------------+
|Year|              Player| Age|Team| GP|   MIN|FGM| FGA|3PM|3PA|FTM|FTA|TOV| PF|ORB|DRB| REB|AST|STL|BLK| PTS|           Franchise|
+----+--------------------+----+----+---+------+---+----+---+---+---+---+---+---+---+---+----+---+---+---+----+--------------------+
|2018|        LeBron James|  33| CLE| 82|3025.8|857|1580|149|406|388|531|347|136| 97|612| 709|747|116| 71|2251| Cleveland Cavaliers|
|2018|        James Harden|  28| HOU| 72|2550.8|651|1449|265|722|624|727|315|169| 41|348| 389|630|126| 50|2191|     Houston Rockets|
|2018|       Anthony Davis|  25| NOP| 75|2727.4|780|1462| 55|162|495|598|162|159|187|645| 832|174|115|193|2110|New Orleans Pelicans|
|2018|   Russell Westbrook|  29| OKC| 80|2913.8|757|1687| 97|326|417|566|381|200|152|652| 804|820|147| 20|2028|Oklahoma City Thu...|
|2018|Giannis Antetokou...|  23| MIL| 75|2756.2|742|1402| 43|140|487|

In [13]:
# adicionar FG%, eFG%, 3P%

df_aux = dataFrame_aux.withColumn(    
    "FG%",
    col("FGM") / col("FGA")
) \
                .withColumn(    
    "eFG%",
    (col("FGM") + 0.5 * col("3PM")) / col("FGA")
) \
                .withColumn(    
    "3P%",
    col("3PM") / col("3PA")
)

In [14]:
# igualar os nomes do df_aux aos do principal

df_aux = df_aux.withColumnRenamed('Team', 'Tm') \
                .withColumnRenamed('Franchise', 'Team') \
                .withColumnRenamed('GP', 'G') \
                .withColumnRenamed('MIN', 'MP') \
                .withColumnRenamed('FGM', 'FG') \
                .withColumnRenamed('3PM', '3P') \
                .withColumnRenamed('FTM', 'FT') \
                .withColumnRenamed('GP', 'G') \
                .withColumnRenamed('REB', 'TRB')

In [15]:
# reordenar df para ficar igual ao principal - prob não é necessário mas wtv
df_aux = df_aux.select("Year","Player","Age","Tm","Team","G","MP","FG","FGA","FG%","3P","3PA","3P%","FT","FTA","TOV","PF","ORB","DRB","TRB","AST","STL","BLK","PTS")

In [18]:
df_aux.printSchema()
#df_aux.show(truncate=False)
#df_aux.toPandas()

root
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: double (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- DRB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- PTS: string (nullable = true)



In [16]:
# EDIÇÃO DO DATAFRAME PRINCIPAL

In [17]:
# apagar o indice porque o toPandas já mostra indice (honestamente é porque facilita a alteração de baixo lmao)
df = dataFrame.drop('#')

# apagar blanl e blank2 porque sao colunas com todas as linhas null (começo a pensar que este dataset não é muito bom xd)
df = df.drop('blanl')
df = df.drop('blank2')

In [18]:
# apagar as 67 linhas que estão em branco
df = df.na.drop("all")

In [19]:
# team faz mais sentido do que franchise
df = df.withColumnRenamed('Franchise', 'Team')

### edicao aux_historical

In [20]:
df_hist.printSchema()
df_hist.toPandas()

NameError: name 'df_hist' is not defined

In [21]:
df_hist = dataFrame_aux_hist.withColumn(    
    "Wins",
    dataFrame_aux_hist.Record.substr(1, 2)
) \
.withColumn(    
    "Losses",
    dataFrame_aux_hist.Record.substr(4, 2)
)

In [22]:
df_hist = df_hist.withColumn(    
    "Year_fixed",
    df_hist.Year.substr(6, 2).cast(IntegerType())
)

In [23]:
df_hist20 = df_hist.filter("Year_fixed < 50")

In [24]:
df_hist19 = df_hist.filter("Year_fixed > 50")

In [25]:
from pyspark.sql import functions as func

df_hist20 = df_hist20.withColumn(    
    "Year_fixed",
    func.concat(lit("20"),func.col('Year_fixed'))
)

df_hist19 = df_hist19.withColumn(    
    "Year_fixed",
    func.concat(lit("19"),func.col('Year_fixed'))
)

In [26]:
df_hist_final = df_hist20.unionByName(df_hist19, allowMissingColumns = True)

In [27]:
df_hist_final = df_hist_final.select("Year_fixed","Tm","Wins","Losses","Winning Percentage")

In [28]:
df_hist_final = df_hist_final.withColumnRenamed("Tm","TmA")

In [32]:
df_hist_final.printSchema()
df_hist_final.toPandas()

root
 |-- Year_fixed: string (nullable = true)
 |-- TmA: string (nullable = true)
 |-- Wins: string (nullable = true)
 |-- Losses: string (nullable = true)
 |-- Winning Percentage: string (nullable = true)



,Year_fixed,TmA,Wins,Losses,Winning Percentage
0,2017,BOS,25,15,0.625
1,2016,BOS,48,34,0.585
2,2015,BOS,40,42,0.488
3,2014,BOS,25,57,0.305
4,2013,BOS,41,40,0.506
...,...,...,...,...,...
1404,1966,BAL,38,42,0.475
1405,1965,BAL,37,43,0.463
1406,1964,BAL,31,49,0.388
1407,1963,CHZ,25,55,0.313


In [33]:
### juncao do hist ao complete

# JUNCAO DOS DOIS DATAFRAMES

In [29]:
df_complete = df.unionByName(df_aux, allowMissingColumns = True)

In [30]:
### juncao do hist ao complete

In [31]:
cond = [df_complete.Year == df_hist_final.Year_fixed, df_complete.Tm == df_hist_final.TmA]

df_complete = df_complete.join(df_hist_final, cond, 'leftouter')

In [32]:
df_complete.filter("Wins > 0").toPandas()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,TOV,PF,PTS,Abbreviation,Team,Year_fixed,TmA,Wins,Losses,Winning Percentage
0,1951,Vince Boryla,SF,23,NYK,66,None,None,None,0.485,...,None,244,982,NYK,New York Knicks,1951,NYK,36,30,0.545
1,1951,Harry Boykoff,C,28,BOS,32,None,None,None,0.436,...,None,130,201,BOS,Boston Celtics,1951,BOS,39,30,0.565
2,1951,Bill Calhoun,SG,23,ROC,66,None,None,None,0.421,...,None,87,511,None,None,1951,ROC,41,27,0.603
3,1951,Nat Clifton*,PF,28,NYK,65,None,None,None,0.364,...,None,269,562,NYK,New York Knicks,1951,NYK,36,30,0.545
4,1951,Jack Coleman,PF,26,ROC,67,None,None,None,0.463,...,None,193,764,None,None,1951,ROC,41,27,0.603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,2017,Thaddeus Young,PF,28,IND,74,74,2237,14.9,0.562,...,96,135,814,IND,Indiana Pacers,2017,IND,22,21,0.512
13101,2017,Tyler Zeller,C,27,BOS,51,5,525,13,0.508,...,20,61,178,BOS,Boston Celtics,2017,BOS,25,15,0.625
13102,2017,Stephen Zimmerman,C,20,ORL,19,0,108,7.3,0.346,...,3,17,23,ORL,Orlando Magic,2017,ORL,18,28,0.391
13103,2017,Paul Zipser,SF,22,CHI,44,18,843,6.9,0.503,...,40,78,240,CHI,Chicago Bulls,2017,CHI,21,23,0.477


In [33]:
df_complete = df_complete.withColumn("Year", df_complete["Year"].cast(StringType())) 
df_complete = df_complete.withColumn("Player", df_complete["Player"].cast(StringType()))
df_complete = df_complete.withColumn("Pos", df_complete["Pos"].cast(StringType())) 
df_complete = df_complete.withColumn("Age", df_complete["Age"].cast(IntegerType()))
df_complete = df_complete.withColumn("Tm", df_complete["Tm"].cast(StringType())) 
df_complete = df_complete.withColumn("G", df_complete["G"].cast(IntegerType())) 
df_complete = df_complete.withColumn("GS",df_complete["GS"].cast(IntegerType())) 
df_complete = df_complete.withColumn("MP", df_complete["MP"].cast(IntegerType())) 
df_complete = df_complete.withColumn("PER", df_complete["PER"].cast(DoubleType()))
df_complete = df_complete.withColumn("TSP", df_complete["TS%"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PAr", df_complete["3PAr"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTr", df_complete["FTr"].cast(DoubleType()))
df_complete = df_complete.withColumn("ORBP", df_complete["ORB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("DRBP", df_complete["DRB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("TRBP", df_complete["TRB%"].cast(DoubleType()))
df_complete = df_complete.withColumn("ASTP", df_complete["AST%"].cast(DoubleType()))
df_complete = df_complete.withColumn("STLP", df_complete["STL%"].cast(DoubleType()))
df_complete = df_complete.withColumn("BLKP", df_complete["BLK%"].cast(DoubleType()))
df_complete = df_complete.withColumn("TOVP", df_complete["TOV%"].cast(DoubleType()))
df_complete = df_complete.withColumn("USGP", df_complete["USG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("OWS", df_complete["OWS"].cast(DoubleType()))
df_complete = df_complete.withColumn("DWS", df_complete["DWS"].cast(DoubleType()))
df_complete = df_complete.withColumn("WS", df_complete["WS"].cast(DoubleType()))
df_complete = df_complete.withColumn("WS48", df_complete["WS/48"].cast(DoubleType()))
df_complete = df_complete.withColumn("OBPM", df_complete["OBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("DBPM", df_complete["DBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("BPM", df_complete["BPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("VORP", df_complete["VORP"].cast(DoubleType()))
df_complete = df_complete.withColumn("FG", df_complete["FG"].cast(DoubleType()))
df_complete = df_complete.withColumn("OBPM", df_complete["OBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("DBPM", df_complete["DBPM"].cast(DoubleType()))
df_complete = df_complete.withColumn("VORP", df_complete["VORP"].cast(DoubleType()))
df_complete = df_complete.withColumn("FG", df_complete["FG"].cast(DoubleType()))
df_complete = df_complete.withColumn("FGA", df_complete["FGA"].cast(DoubleType()))
df_complete = df_complete.withColumn("FGP", df_complete["FG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("3P", df_complete["3P"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PA", df_complete["3PA"].cast(DoubleType()))
df_complete = df_complete.withColumn("3PP", df_complete["3P%"].cast(DoubleType()))
df_complete = df_complete.withColumn("2P", df_complete["2P"].cast(DoubleType()))
df_complete = df_complete.withColumn("2PA", df_complete["2PA"].cast(DoubleType()))
df_complete = df_complete.withColumn("2PP", df_complete["2P%"].cast(DoubleType()))
df_complete = df_complete.withColumn("eFGP", df_complete["eFG%"].cast(DoubleType()))
df_complete = df_complete.withColumn("FT", df_complete["FT"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTA", df_complete["FTA"].cast(DoubleType()))
df_complete = df_complete.withColumn("FTP", df_complete["FT%"].cast(DoubleType()))
df_complete = df_complete.withColumn("ORB", df_complete["ORB"].cast(DoubleType()))
df_complete = df_complete.withColumn("DRB", df_complete["DRB"].cast(DoubleType()))
df_complete = df_complete.withColumn("TRB", df_complete["TRB"].cast(DoubleType()))
df_complete = df_complete.withColumn("AST", df_complete["AST"].cast(DoubleType()))
df_complete = df_complete.withColumn("STL", df_complete["STL"].cast(DoubleType()))
df_complete = df_complete.withColumn("BLK", df_complete["BLK"].cast(DoubleType()))
df_complete = df_complete.withColumn("TOV", df_complete["TOV"].cast(DoubleType()))
df_complete = df_complete.withColumn("PF", df_complete["PF"].cast(DoubleType()))
df_complete = df_complete.withColumn("PTS", df_complete["PTS"].cast(DoubleType()))
#df_complete = df_complete.withColumn("Abbreviation", df_complete["Abbreviation"].cast(StringType()))
df_complete = df_complete.withColumn("Team", df_complete["Team"].cast(StringType()))

df_complete = df_complete.withColumn("Wins", df_complete["Wins"].cast(IntegerType()))
df_complete = df_complete.withColumn("Losses", df_complete["Losses"].cast(IntegerType()))
df_complete = df_complete.withColumn("WP", df_complete["Winning Percentage"].cast(DoubleType()))

In [ ]:
df_complete.printSchema()
#df_complete.show(truncate=False)
#df_complete.toPandas()
#topandas usar apenas para mostrar os dados em tabela

In [34]:
df_complete \
    .select("Year","Player","Pos","Age","Tm","Team","G","GS","Wins","Losses","WP","MP","PER","TSP","3PAr","FTr","ORBP","DRBP","TRBP","ASTP","STLP","BLKP","TOVP","USGP","OWS","DWS","WS","WS48","OBPM","DBPM","BPM","VORP","FG","FGA","FGP","3P","3PA","3PP","2P","2PA","2PP","eFGP","FT","FTA","FTP","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS") \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/silver/stats")

22/01/13 22:59:48 WARN NettyRpcEnv: Ignored message: HeartbeatResponse(false)   
22/01/13 22:59:57 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:996)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.s